In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

<b> Загрузка и предобработка исходных данных

In [2]:
data = pd.read_csv("../../Практические материалы/Lab 1. Linear and logistic regression/mnist/mnist_train.csv")
data_test = pd.read_csv("../../Практические материалы/Lab 1. Linear and logistic regression/mnist/mnist_test.csv")

In [3]:
X = data.iloc[:,1:].values
X_test = data_test.iloc[:,1:].values

In [4]:
mean = np.mean(X)
std = np.std(X)

X = (X - mean)/std
X_test = (X_test - mean)/std

In [5]:
m = len(data)
n = data.shape[1]-1

In [6]:
Y = np.zeros([m,10])

In [7]:
for i in range(m):
    Y[i,data.label.values[i]] = 1

In [8]:
Y

array([[0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.]])

<b> Алгоритм обратного распространения ошибки

In [10]:
def sigmoid(z):
    return 1/(1+np.exp(-z))

In [11]:
def tanh(z):
    return (np.exp(z)-np.exp(-z))/(np.exp(z)+np.exp(-z))

In [22]:
def forward(W1,W2,W3,B1,B2,B3,X):
    F1 = tanh(np.matmul(X,W1) + B1)
    F2 = sigmoid(np.matmul(F1,W2) + B2)
    H = sigmoid(np.matmul(F2,W3) + B3)
    return F1,F2,H

In [23]:
def backpropagation(W1,W2,W3,B1,B2,B3,H,F2,F1,X):
    S3 = H - Y

    dW3 = np.matmul(np.transpose(F2),S3)
    dB3 = np.sum(S3, axis = 0)

    S2 = np.matmul(S3,np.transpose(W3))*F2*(1-F2)

    dW2 = np.matmul(np.transpose(F1),S2)
    dB2 = np.sum(S2, axis = 0)

    S1 = np.matmul(S2,np.transpose(W2))*(1-F1**2)

    dW1 = np.matmul(np.transpose(X),S1)
    dB1 = np.sum(S1, axis = 0)
    
    return dW3,dW2,dW1,dB3,dB2,dB1

In [24]:
k1 = 100
k2 = 50
k3 = 10

W1 = np.random.rand(n,k1) - 0.5
W2 = np.random.rand(k1,k2) - 0.5
W3 = np.random.rand(k2,k3) - 0.5
B1 = np.zeros([1,k1])
B2 = np.zeros([1,k2])
B3 = np.zeros([1,k3])
#dropout1 = np.round(np.random.rand(1,k1))
#dropout2 = np.round(np.random.rand(1,k2))

alpha = 0.00001

for i in range(10000):
    F1,F2,H = forward(W1,W2,W3,B1,B2,B3,X)
    
    J = -np.sum((Y*np.log(H) + (1-Y)*np.log(1-H)))
    print(i,J)
    
    dW3,dW2,dW1,dB3,dB2,dB1 = backpropagation(W1,W2,W3,B1,B2,B3,H,F2,F1,X)
    break
    W3 = W3 - alpha*dW3
    W2 = W2 - alpha*dW2
    W1 = W1 - alpha*dW1
    B3 = B3 - alpha*dB3
    B2 = B2 - alpha*dB2
    B1 = B1 - alpha*dB1

0 510865.7475493406


<b> Численная проверка градиента

In [25]:
F1,F2,H = forward(W1,W2,W3,B1,B2,B3,X)

J1 = -np.sum((Y*np.log(H) + (1-Y)*np.log(1-H)))

W1[0,9] += 0.0001

F1,F2,H = forward(W1,W2,W3,B1,B2,B3,X)

J2 = -np.sum((Y*np.log(H) + (1-Y)*np.log(1-H)))

W1[0,9] -= 0.0001

print((J2-J1)/0.0001)

63.99858451914042


In [26]:
print(dW1[0,9])

63.998465729436155


<b> Расчёт предсказаний для тестовой выборки

In [37]:
_,_,pred = forward(W1*dropout1,W2*dropout2,W3*dropout3,B1,B2,B3,X_test)

In [38]:
pred

array([[2.48374242e-03, 2.21395900e-03, 2.83102182e-05, ...,
        9.99483950e-01, 5.29819257e-05, 1.51114354e-03],
       [1.95513358e-03, 1.61850360e-05, 9.89319278e-01, ...,
        1.34639176e-04, 1.64916875e-03, 1.06544445e-06],
       [5.69135380e-06, 9.93854267e-01, 2.72311614e-03, ...,
        4.24032586e-03, 3.86161621e-04, 7.57513044e-06],
       ...,
       [1.29084950e-03, 2.38206126e-05, 9.45360903e-04, ...,
        8.92821970e-04, 8.87328736e-03, 2.26559810e-03],
       [4.21069465e-04, 1.10188393e-02, 5.07410751e-05, ...,
        8.97795556e-04, 2.71929559e-02, 5.03960462e-05],
       [1.19111082e-02, 5.91523591e-04, 2.38543479e-03, ...,
        3.29661469e-05, 1.68326475e-05, 1.81673843e-05]])

In [39]:
pred = np.argmax(pred,axis=1)

In [40]:
pred

array([7, 2, 1, ..., 4, 5, 6])

In [41]:
data_test.label

0       7
1       2
2       1
3       0
4       4
       ..
9995    2
9996    3
9997    4
9998    5
9999    6
Name: label, Length: 10000, dtype: int64

In [42]:
np.mean(data_test.label == pred)

0.9575

In [43]:
data_test['pred'] = pred

In [44]:
for i in range(10):
    temp = data_test[data_test.label == i]
    print(np.mean(np.mean(temp.label == temp.pred)))

0.9877551020408163
0.9859030837004406
0.9505813953488372
0.9574257425742574
0.955193482688391
0.9394618834080718
0.965553235908142
0.9455252918287937
0.9455852156057495
0.9375619425173439
